In [1]:
from bs4 import BeautifulSoup
import asyncio
import time
import threading
from aiohttp import ClientSession
import re
from tqdm import tqdm

async def get(url):
  async with ClientSession() as session:
    async with session.get(url) as response:
      return await response.read()

def parse_table(table):
  return [[el.text for el in row.find_all(['th', 'td'])] for row in table.find_all('tr')]

base_url = 'http://profiles.asee.org'
loop = asyncio.get_event_loop()
res = loop.run_until_complete(get(base_url))
base_soup = BeautifulSoup(res, 'html.parser')

schools = base_soup.find('table').find_all('td', class_='school_name')
profile_links = [base_url+school.a['href'] for school in schools]

school_ids = []
profile_res = loop.run_until_complete(asyncio.gather(*[get(link) for link in profile_links]))

for profile in tqdm(profile_res):
  profile_soup = BeautifulSoup(profile, 'html.parser')
  profile_ids = profile_soup.find('select', id='survey_id').find_all('option')
  profile_ids = [pid['value'] for pid in profile_ids]
  school_ids.append(profile_ids)

fetch = []
profile_base = 'http://profiles.asee.org/profiles/{}/print_all'
for school in tqdm(school_ids):
  fetch.append(get(profile_base.format(school[0])))

# main fetch sequence
school_res = loop.run_until_complete(asyncio.gather(*fetch))

100%|██████████| 295/295 [00:00<00:00, 264837.26it/s]


In [6]:
school_res[0]

b'<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN"\n       "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\n<html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en" lang="en">\n<head>\n  <meta http-equiv="content-type" content="text/html;charset=UTF-8" />\n  <title>Air Force Institute of Technology - 2018</title>\n    <link href="/stylesheets/asee_main.css" media="screen" rel="stylesheet" type="text/css" />\n    <link href="/stylesheets/asee_main-print.css" media="print" rel="stylesheet" type="text/css" />\n  <link href="/stylesheets/print_packaged.css?1465226486" media="screen" rel="stylesheet" type="text/css" />\n  <!--[if lte IE 6]>\n    <link rel="stylesheet" href="/stylesheets/ie.css" type="text/css" media="screen, projection" /> \n  <![endif]-->\n</head>\n<body>          \n  <div class="container">\n\t<div id="content">\n\t\t \n\t\t<div class="box">\n\t\t\t<div class="box-outer">\n\t\t\t\t<div class="box-inner">  \n\t\t\t\t\t<span class="top"></span>\n\

In [17]:
data = {}
for school_html in tqdm(school_res):
  soup = BeautifulSoup(school_html, 'html.parser')
  
  # initialize values
  institution_type = 'none'
  undergrad_enroll = 'none'
  graduate_enroll = 'none'
  description = 'none'
  teaching_tenure = 'none'
  tuition = 'none'
  sat_scores = 'none'
  act_scores = 'none'
  enrollment = 'none'
  
  # extract attributes
  name = soup.find('h1').text[:-7]
  print('School: {}'.format(name))

  try:
    institution_type = soup.find('h3', text='General Information').find_next_sibling('table').find_all('td')[1].text
  except AttributeError:
    print('bad institution_type for {}'.format(name))
    
  try:
    undergrad_enroll = soup.find('p', class_='highlighted', text='Total Enrollment').find_next_sibling('table').find_all('td')[1].text.replace(',','')
    graduate_enroll = soup.find('p', class_='highlighted', text='Total Enrollment').find_next_sibling('table').find_all('td')[3].text.replace(',','')
  except AttributeError:
    print('bad enrollment for {}'.format(name))
  
  try:
    description = soup.find('p', class_='highlighted', text='Engineering College Description and Special Characteristics').find_next_sibling('p').text
  except AttributeError:
    print('bad description for {}'.format(name))
      
  try: 
    teaching_tenure_table = soup.find('p', class_='highlighted', text='Teaching, Tenure-Track: Full Professor Profiles').find_next_sibling('table')
    teaching_tenure = parse_table(teaching_tenure_table)
  except AttributeError:
    print('bad teaching_tenure for {}'.format(name))

  try:
    tuition_list = parse_table(soup.find('h3', text='Expenses & Financial Aid').find_next_sibling('table'))[1]
    tuition = tuition_list[1:3] if len(tuition_list) > 2 else tuition_list[1]
    tuition = [t.replace('$','').replace(',','') for t in tuition]
  except AttributeError:
    print('bad tuition for {}'.format(name))

  try:
    test_scores = soup.find('p', class_='highlighted', text=' Newly Enrolled Test Scores').find_next_sibling('table')
    sat_scores = parse_table(test_scores)
    act_scores = parse_table(test_scores.find_next_sibling('table'))
  except AttributeError:
    print('bad test_scores for {}'.format(name))

  try: 
    enrollment_table = soup.find('p', class_='highlighted', text='Degrees By Ethnicity & Gender').find_next_sibling('table')
    enrollment = parse_table(enrollment_table)
    enrollment = [e for e in enrollment if ('Note' not in e[0])]
  except AttributeError:
    print('bad enrollment for {}'.format(name))

  data[name] = {
    'type': institution_type,
    'undergrad_enroll': undergrad_enroll,
    'graduate_enroll': graduate_enroll,
    'description': description,
    'teaching_tenure': teaching_tenure,
    'tuition': tuition,
    'sat_scores': sat_scores,
    'act_scores': act_scores,
    'enrollment': enrollment
  }


  0%|          | 1/295 [00:00<01:59,  2.46it/s]

School: Air Force Institute of Technology
bad test_scores for Air Force Institute of Technology



  1%|          | 2/295 [00:00<01:46,  2.76it/s]

School: The University of Akron, Engineering & Science Technology Dept.
bad description for The University of Akron, Engineering & Science Technology Dept.



  1%|          | 3/295 [00:01<01:48,  2.70it/s]

...

School: University of Wyoming



100%|█████████▉| 294/295 [06:38<00:00,  1.29it/s]

School: Yale School of Engineering & Applied Science



100%|██████████| 295/295 [06:38<00:00,  1.66it/s]

School: York College of Pennsylvania
